In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

keys = ['采集时间', '水温', 'pH', '溶解氧', '电导率', '浊度', '高锰酸盐指数','氨氮', '总磷', '总氮']
en_keys = ['WaterTemperature', 'PH' ,'dissolved oxygen', 'Conductivity','Turbidity','PermanganateIndex',
        'AmmoniaNitrogen','TP','TN', 'humidity','room temperature','chlorophyll','Algae density']
# limits = [(5, 30), (5.0, 9), (1, 15), (50, 500), (0, 1500), (0, 15), (0, 0.5), (0, 0.3), (0, 5)]

np.set_printoptions(formatter = {'float': '{:.2e}'.format})


In [ ]:
df = pd.read_excel('./origional_data/泸沽湖邛海鲁班水库水质数据/原始查询/原始查询（鲁班岛-鲁班水库）.xls', 
                   header=2, usecols=keys, index_col=0)
df = df.loc[::-1]
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
df.replace(to_replace= '--', value = np.nan, inplace=True)
df = df.astype('float64')
df[df < 0] = np.nan
# df = df.drop(df[df.index <= '2019-02-07'].index)
# for date in find_miss(df): print(date.date())
# # print(find_miss(df))
df = df.resample('4H', 'index').mean()
# print(df['氨氮'].index[df['氨氮'].isna()])

df = remove_outliers(df, standard_deviation, 25)


df = patch_up(df, 14)

# df = df.interpolate(limit_direction='both', method='linear', axis=1)

# # #  Denoise
df = smooth(df, 3)

# # # Rescale
# # # df = (df - df.min()) / (df.max() - df.min())
# df = (df - df.mean())/df.std()
# df.index.max()

In [ ]:
print(df.index.max(), df.index.min())
index = df.index[df.isna().any(axis=1)]
print(df.loc[index])

In [ ]:
index_nums = len(df.keys())
print(df.keys())
l, h = 18, 3
fig, axis = plt.subplots(index_nums, 1, figsize=(l, h*index_nums), constrained_layout=True)
for i in range(index_nums):
    name = en_keys[i]
    df.plot(y=df.keys()[i], ax=axis[i])
    axis[i].set_title(name, fontsize=20)
    axis[i].set_xlabel('', fontsize=15)
    axis[i].set_ylabel('', fontsize=15)
    
    axis[i].legend([name], fontsize=15)

In [ ]:
df['水温'].values.shape[0]

In [ ]:
figure, axis = plt.subplots(3, 1, figsize=(30, 10))
name = keys[5]
vals = df[name].values.copy()
print(f'{name}, {len(vals)}')
print(vals)
X = np.linspace(0, 1, len(vals))
axis[0].plot(X, vals)
axis[0].set_title('UnSmooth')
smooth_vals = pd.Series(vals).ewm(span=30).mean()
axis[1].plot(X, smooth_vals)
axis[1].set_title('Smooth')
scaled_vals = (smooth_vals - smooth_vals.min()) / smooth_vals.max()
axis[2].plot(X, scaled_vals)
axis[2].set_title('scaled')


plt.show()

In [ ]:
from pathlib import Path
p = Path("./data/")
for csv in p.glob("*.csv"):
    print(csv.stem)

In [11]:
df = pd.read_csv('./data/luban.csv', index_col=0)
mean, std = df.loc['mean'].values, df.loc['var'].values
df = df.drop(['mean', 'var'])
df

,水温,pH,溶解氧,电导率,浊度,高锰酸盐指数,氨氮,总磷,总氮
采集时间,,,,,,,,,
2018-06-27 00:00:00,0.137331,-0.831795,-0.134025,0.004660,0.868249,-0.748582,13.011583,-54.766689,1.239575
2018-06-27 04:00:00,0.140101,-0.707907,-0.113788,0.003519,0.411920,-0.748582,18.881817,-54.766689,2.964412
2018-06-27 08:00:00,0.143548,-0.614219,-0.112677,0.002926,0.250781,-0.748582,20.785676,-54.766689,3.523818
2018-06-27 12:00:00,0.155307,0.035774,-0.022002,0.001549,0.161131,-0.748582,20.725505,-54.766689,2.836664
2018-06-27 16:00:00,0.166281,0.565156,0.017160,-0.001039,0.108826,-0.748582,14.580264,-54.766689,2.247173
...,...,...,...,...,...,...,...,...,...
2022-09-22 04:00:00,0.077526,0.776628,-0.268618,-0.002526,0.016413,0.098447,-3.310026,-19.172548,-2.506704
2022-09-22 08:00:00,0.076559,0.734553,-0.286754,-0.002510,0.018813,0.073757,-3.452930,-19.322169,-2.605065
2022-09-22 12:00:00,0.078043,0.766905,-0.273153,-0.002517,0.019867,0.085547,-3.693276,-19.981192,-2.697501


In [12]:
df = df * std + mean
df

,水温,pH,溶解氧,电导率,浊度,高锰酸盐指数,氨氮,总磷,总氮
采集时间,,,,,,,,,
2018-06-27 00:00:00,26.713333,7.731667,7.894333,305.029333,1131.443333,0.000000,0.068429,1.106754e-15,0.727571
2018-06-27 04:00:00,26.841429,7.800000,8.016714,294.526857,544.382286,0.000000,0.090469,1.106754e-15,0.912388
2018-06-27 08:00:00,27.000811,7.851676,8.023432,289.073730,337.078811,0.000000,0.097618,1.106754e-15,0.972328
2018-06-27 12:00:00,27.544514,8.210194,8.571754,276.408229,221.745943,0.000000,0.097392,1.106754e-15,0.898700
2018-06-27 16:00:00,28.051921,8.502187,8.808577,252.591795,154.455493,0.000000,0.074318,1.106754e-15,0.835536
...,...,...,...,...,...,...,...,...,...
2022-09-22 04:00:00,23.948031,8.618829,7.080434,238.910589,35.567828,2.026248,0.007146,1.627363e-02,0.326157
2022-09-22 08:00:00,23.903336,8.595622,6.970763,239.058067,38.654996,1.967186,0.006610,1.620522e-02,0.315618
2022-09-22 12:00:00,23.971939,8.613467,7.053010,238.992175,40.010935,1.995390,0.005707,1.590391e-02,0.305713
